## Association Rules - Old vs. New Algorithm

### Dataset: eCommerce_purchase_history_from_cosmetics_store

#### 1 Investigating Dataset

#### 1.1 Description
Kaggle Dataset: https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-cosmetics-shop <br>
The Dataset is Open-Source <br>
This file contains behavior data for 5 months (Oct 2019 – Feb 2020) from a medium cosmetics online store.<br>

In [14]:
import os
import pandas as pd
import glob

In [15]:
import glob
import os

# setting the path for joining multiple files
files = os.path.join(os.getcwd(), "datasets/eCommerce_purchase_history_from_cosmetics_store/*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
data_raw = pd.concat(map(pd.read_csv, files), ignore_index=True).iloc[:,1:].rename(columns={"user_session":"order_id"})

data_raw


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,order_id
0,2019-12-01 00:17:50 UTC,purchase,4554,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
1,2019-12-01 00:17:50 UTC,purchase,4600,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
2,2019-12-01 00:17:50 UTC,purchase,4607,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
3,2019-12-01 00:17:50 UTC,purchase,4640,1487580011157258342,NaN,runail,0.37,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
4,2019-12-01 00:17:50 UTC,purchase,4689,1487580010796548183,NaN,runail,0.38,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
...,...,...,...,...,...,...,...,...,...
1287002,2020-01-31 23:58:26 UTC,purchase,5730206,1487580005092295511,NaN,NaN,10.32,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287003,2020-01-31 23:58:26 UTC,purchase,5861342,1487580010645553231,NaN,NaN,1.90,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287004,2020-01-31 23:58:26 UTC,purchase,5867185,1487580007910867929,NaN,staleks,11.43,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e
1287005,2020-01-31 23:58:26 UTC,purchase,5877766,1605161575889502297,NaN,NaN,4.76,604470329,6f853f2d-e1aa-49d4-84d8-284566ea1a0e


In [16]:
# Quantity is always 1
grouped_mutliple_items = data_raw.groupby(['order_id', 'product_id']).agg({'user_id': ['count']})

#grouped_mutliple_items[grouped_mutliple_items["quantity"]>1]

grouped_mutliple_items.columns = ["user_id"]
grouped_mutliple_items.rename(columns={"user_id":"quantity"},inplace=True)
grouped_mutliple_items = grouped_mutliple_items.reset_index()
grouped_mutliple_items = grouped_mutliple_items[grouped_mutliple_items["quantity"]>1].sort_values("quantity",ascending=False)

In [17]:
grouped_mutliple_items

,order_id,product_id,quantity
368420,49bc852c-cbc7-4d84-9a2c-8144205b571a,5751422,10
368421,49bc852c-cbc7-4d84-9a2c-8144205b571a,5809910,10
1252940,fb5a2f8f-837c-4e50-9b3d-bdcbdc75315e,5778113,9
368422,49bc852c-cbc7-4d84-9a2c-8144205b571a,5809912,9
368414,49bc852c-cbc7-4d84-9a2c-8144205b571a,5304,9
...,...,...,...
435978,570ac7f0-7bc2-7587-5c70-388c76d73451,5809303,2
435979,570ac7f0-7bc2-7587-5c70-388c76d73451,5841799,2
435980,570ac7f0-7bc2-7587-5c70-388c76d73451,5856952,2
435981,570ac7f0-7bc2-7587-5c70-388c76d73451,5879128,2


In [18]:
# Mutiple items per transaction

import seaborn as sns

mutliple_items_per_transaction = grouped_mutliple_items["quantity"].value_counts()
mutliple_items_per_transaction.rename_axis("frequency")

# lenth_of_transactions
mutliple_items_per_transaction = mutliple_items_per_transaction.head(4)

total = mutliple_items_per_transaction.values.sum()

import plotly.express as px
fig = px.bar(mutliple_items_per_transaction, x=mutliple_items_per_transaction.index, y=mutliple_items_per_transaction.values/total,width=400, height=400)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Multiple items per transaction",
    yaxis_tickformat ='0%',
    xaxis_tickformat = ',d',
    yaxis_title="Distribution",
    legend_title="Multiple items per transaction distribution",)

fig.show()


In [19]:
# group by Transaction

import math
import datetime

data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_27340/4105145759.py:12: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,date,price,margin
order_id,,,,
000013d6-68a4-40cf-9452-6577dbfab515,"[5889104, 5878254, 5875502, 5859210, 5739125, ...","[2019-10-23, 2019-10-23, 2019-10-23, 2019-10-2...","[4.76, 5.56, 1.75, 0.48, 4.4, 5.22, 0.48, 5.08...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
00008836-6d96-4580-8527-173d4e1b2889,"[5730402, 5853682, 5839703, 5839663, 5839660, ...","[2019-10-02, 2019-10-02, 2019-10-02, 2019-10-0...","[7.86, 3.63, 1.73, 2.37, 2.37, 1.73, 1.73, 1.4...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
0000d04d-3516-42ea-bdb9-584e92ce619e,"[5809100, 5809101, 5854897, 5888573]","[2020-02-15, 2020-02-15, 2020-02-15, 2020-02-15]","[1.59, 1.59, 0.32, 3.33]","[0.1, 0.1, 0.1, 0.1]"
0001981a-ee7c-4542-bf81-2cea59d963ab,"[5839084, 5788435, 5743913, 5743912, 5677989, ...","[2019-10-05, 2019-10-05, 2019-10-05, 2019-10-0...","[4.86, 0.79, 3.97, 3.97, 8.73, 1.43]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
000277a9-30da-4bb2-b5ad-96ef24483af8,"[5815567, 5815568, 5815569, 5815570, 5815571]","[2019-12-14, 2019-12-14, 2019-12-14, 2019-12-1...","[0.71, 0.71, 0.71, 0.71, 0.71]","[0.1, 0.1, 0.1, 0.1, 0.1]"
...,...,...,...,...
fffee574-c72f-03cf-4d2a-5188047c4cef,"[5560968, 5857357, 5695534, 5723501, 5734268, ...","[2019-12-24, 2019-12-24, 2019-12-24, 2019-12-2...","[3.0, 2.78, 3.33, 2.62, 3.97, 6.19, 3.33, 3.0,...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
ffff78e3-1e8f-4eda-bb42-95749ab4f42e,"[5686083, 5710585, 5776129, 5776130, 5776132]","[2020-02-21, 2020-02-21, 2020-02-21, 2020-02-2...","[0.79, 1.43, 2.7, 1.98, 2.7]","[0.1, 0.1, 0.1, 0.1, 0.1]"
ffff8529-58c1-4ec8-933e-77f535718061,"[5875389, 5859403, 5839651, 5752164, 5727536, ...","[2019-10-13, 2019-10-13, 2019-10-13, 2019-10-1...","[2.05, 2.37, 2.37, 3.97, 2.7, 1.73]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"


In [20]:
# Length of transactions

import seaborn as sns

length_of_transactions = data["product_id"].apply(len).value_counts()
length_of_transactions = length_of_transactions.sort_index(ascending=True).head(20) # For graph only shop top 20
length_of_transactions.rename_axis("frequency")


total = length_of_transactions.sum()

import plotly.express as px
fig = px.bar(length_of_transactions, x=length_of_transactions.index, y=length_of_transactions.values/total)
fig.update_traces(width=0.5)
fig.update_layout(
    xaxis_title="Number of items per transaction",
    yaxis_tickformat =',.0%',
    yaxis_title="Distribution (%)",
    legend_title="Transaction Length Dsitribution",)

fig.show()

In [21]:
unique_items = data_raw[["product_id","category_id","category_code","brand","price"]].drop_duplicates(subset = "product_id").astype("str")


unique_items

,product_id,category_id,category_code,brand,price
0,4554,1487580010796548183,nan,runail,0.38
1,4600,1487580010796548183,nan,runail,0.38
2,4607,1487580010796548183,nan,runail,0.38
3,4640,1487580011157258342,nan,runail,0.37
4,4689,1487580010796548183,nan,runail,0.38
...,...,...,...,...,...
1285867,5913479,1542195323827388674,nan,dermacol,13.97
1285868,5915330,1542195323827388674,nan,dermacol,19.05
1286757,5923045,1487580007675986893,nan,nan,1.43
1286948,5889114,2134354342373753638,nan,nan,3.49


In [22]:
unique_items["price"].astype("float").median()
unique_items["price"].astype("float").mean()

7.056675331682017

In [64]:
data_raw["date"] = pd.to_datetime(data_raw['event_time']).dt.date
max_date = data_raw["date"].max()
data_raw["margin"] = 0.1
data_raw["profit"] = data_raw["price"] * data_raw["margin"]
max_profit = data_raw["profit"].max()

data = data_raw.groupby("order_id",dropna=True)["product_id","date","price","margin"].agg(lambda x: list(x))

data

C:\Users\david\AppData\Local\Temp/ipykernel_27340/16707732.py:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,date,price,margin
order_id,,,,
000013d6-68a4-40cf-9452-6577dbfab515,"[5889104, 5878254, 5875502, 5859210, 5739125, ...","[2019-10-23, 2019-10-23, 2019-10-23, 2019-10-2...","[4.76, 5.56, 1.75, 0.48, 4.4, 5.22, 0.48, 5.08...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
00008836-6d96-4580-8527-173d4e1b2889,"[5730402, 5853682, 5839703, 5839663, 5839660, ...","[2019-10-02, 2019-10-02, 2019-10-02, 2019-10-0...","[7.86, 3.63, 1.73, 2.37, 2.37, 1.73, 1.73, 1.4...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ..."
0000d04d-3516-42ea-bdb9-584e92ce619e,"[5809100, 5809101, 5854897, 5888573]","[2020-02-15, 2020-02-15, 2020-02-15, 2020-02-15]","[1.59, 1.59, 0.32, 3.33]","[0.1, 0.1, 0.1, 0.1]"
0001981a-ee7c-4542-bf81-2cea59d963ab,"[5839084, 5788435, 5743913, 5743912, 5677989, ...","[2019-10-05, 2019-10-05, 2019-10-05, 2019-10-0...","[4.86, 0.79, 3.97, 3.97, 8.73, 1.43]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
000277a9-30da-4bb2-b5ad-96ef24483af8,"[5815567, 5815568, 5815569, 5815570, 5815571]","[2019-12-14, 2019-12-14, 2019-12-14, 2019-12-1...","[0.71, 0.71, 0.71, 0.71, 0.71]","[0.1, 0.1, 0.1, 0.1, 0.1]"
...,...,...,...,...
fffee574-c72f-03cf-4d2a-5188047c4cef,"[5560968, 5857357, 5695534, 5723501, 5734268, ...","[2019-12-24, 2019-12-24, 2019-12-24, 2019-12-2...","[3.0, 2.78, 3.33, 2.62, 3.97, 6.19, 3.33, 3.0,...","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"
ffff78e3-1e8f-4eda-bb42-95749ab4f42e,"[5686083, 5710585, 5776129, 5776130, 5776132]","[2020-02-21, 2020-02-21, 2020-02-21, 2020-02-2...","[0.79, 1.43, 2.7, 1.98, 2.7]","[0.1, 0.1, 0.1, 0.1, 0.1]"
ffff8529-58c1-4ec8-933e-77f535718061,"[5875389, 5859403, 5839651, 5752164, 5727536, ...","[2019-10-13, 2019-10-13, 2019-10-13, 2019-10-1...","[2.05, 2.37, 2.37, 3.97, 2.7, 1.73]","[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]"


#### 3.1.1 Classical Association Rules

In [37]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#data = data[data["product_id"].apply(len)<5]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.01,
    maxSupRatio=1,
    minConf=0,
    item_col=1
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,percSupAntecedent,consequent,sup_consequent,percSupConsequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[],NA,NA,[5809910],7458,0.047925,[5809910],7458,0.047925,NA,NA,NA
1,[],NA,NA,[5854897],4593,0.029515,[5854897],4593,0.029515,NA,NA,NA
2,[],NA,NA,[5700037],3623,0.023282,[5700037],3623,0.023282,NA,NA,NA
3,[],NA,NA,[5802432],3486,0.022401,[5802432],3486,0.022401,NA,NA,NA
4,[],NA,NA,[5751422],3457,0.022215,[5751422],3457,0.022215,NA,NA,NA
5,[],NA,NA,[5809912],3272,0.021026,[5809912],3272,0.021026,NA,NA,NA
6,[],NA,NA,[5815662],3206,0.020602,[5815662],3206,0.020602,NA,NA,NA
7,[],NA,NA,[5304],3102,0.019934,[5304],3102,0.019934,NA,NA,NA
8,[],NA,NA,[5751383],2912,0.018713,[5751383],2912,0.018713,NA,NA,NA
9,[],NA,NA,[5849033],2753,0.017691,[5849033],2753,0.017691,NA,NA,NA


In [38]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.
from IPython.display import HTML

rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

#rules_labeled = rules_labeled.drop('rules_index', axis=1)

rules_labeled_new = rules_labeled[["rules_index","antecedent&consequent","sup_ant&cons","category_id","price"]]

HTML(rules_labeled_new.to_html(index=False))

rules_index,antecedent&consequent,sup_ant&cons,category_id,price
0,5809910,7458,1602943681873052386,5.24
1,5854897,4593,1487580009445982239,0.32
2,5700037,3623,1487580009286598681,0.4
3,5802432,3486,1487580009286598681,0.32
4,5751422,3457,1487580005268456287,10.95
5,5809912,3272,1602943681873052386,5.24
6,5815662,3206,1487580006317032337,0.92
7,5304,3102,1487580009471148064,0.32
8,5751383,2912,1487580005092295511,10.32
9,5849033,2753,1487580005092295511,10.32


#### 3.1.2 Classical Association Rules (Limitations on transaction length as at least 2)

In [73]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

data = data[data["product_id"].apply(len)<20]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0025,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    minTransactionLength=2,
    maxTransactionLength=10
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,percSupAntecedent,consequent,sup_consequent,percSupConsequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement
0,[5809910],6244,0.043521,[5809912],2596,0.018094,"[5809910, 5809912]",1180,0.008225,0.188981,10.444281,0.170887
1,[5751422],2860,0.019934,[5751383],2441,0.017014,"[5751383, 5751422]",837,0.005834,0.292657,17.201082,0.275643
2,[5809910],6244,0.043521,[5809911],1412,0.009842,"[5809911, 5809910]",756,0.005269,0.121076,12.302357,0.111235
3,[5809910],6244,0.043521,[5816170],1398,0.009744,"[5816170, 5809910]",569,0.003966,0.091127,9.352039,0.081383
4,[5809912],2596,0.018094,[5809911],1412,0.009842,"[5809911, 5809912]",546,0.003806,0.210324,21.370633,0.200482
5,[5751422],2860,0.019934,[5849033],2288,0.015947,"[5849033, 5751422]",529,0.003687,0.184965,11.598391,0.169018
6,[5809912],2596,0.018094,[5816170],1398,0.009744,"[5816170, 5809912]",411,0.002865,0.158320,16.247782,0.148576
7,"[5809910, 5809912]",1180,0.008225,[5809911],1412,0.009842,"[5809911, 5809910, 5809912]",381,0.002656,0.322881,32.807444,0.313040


In [74]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled_new = rules_labeled[["rules_index","antecedent&consequent","sup_ant&cons","category_id","brand","price"]]

HTML(rules_labeled_new.to_html(index=False))

rules_index,antecedent&consequent,sup_ant&cons,category_id,brand,price
0,5809910,1180,1602943681873052386,grattol,5.24
0,5809912,1180,1602943681873052386,grattol,5.24
1,5751422,837,1487580005268456287,uno,10.95
1,5751383,837,1487580005092295511,uno,10.32
2,5809910,756,1602943681873052386,grattol,5.24
2,5809911,756,1602943681873052386,grattol,5.24
3,5809910,569,1602943681873052386,grattol,5.24
3,5816170,569,1602943681873052386,grattol,5.24
4,5809912,546,1602943681873052386,grattol,5.24
4,5809911,546,1602943681873052386,grattol,5.24


#### 3.2 Profit based Association Rules (No limitations on transaction length)

In [55]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"

# data = data[data["product_id"].apply(len)<=5]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.0038,
    maxSupRatio=1,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules


,antecedent,sup_antecedent,percSupAntecedent,consequent,sup_consequent,percSupConsequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[],NA,NA,[5560754],365,0.002346,[5560754],365,0.002346,NA,NA,NA,71726.72,0.011299,0,71726.72,71726.72,0
1,[],NA,NA,[5809910],7458,0.047925,[5809910],7458,0.047925,NA,NA,NA,39202.47,0.006176,0,39202.47,39202.47,0
2,[],NA,NA,[5751422],3457,0.022215,[5751422],3457,0.022215,NA,NA,NA,38229.81,0.006022,0,38229.81,38229.81,0
3,[],NA,NA,[5751383],2912,0.018713,[5751383],2912,0.018713,NA,NA,NA,30138.29,0.004748,0,30138.29,30138.29,0
4,[],NA,NA,[5850281],209,0.001343,[5850281],209,0.001343,NA,NA,NA,29283.62,0.004613,0,29283.62,29283.62,0
5,[],NA,NA,[5849033],2753,0.017691,[5849033],2753,0.017691,NA,NA,NA,28416.78,0.004476,0,28416.78,28416.78,0
6,[],NA,NA,[5792800],2683,0.017241,[5792800],2683,0.017241,NA,NA,NA,27764.36,0.004374,0,27764.36,27764.36,0
7,[],NA,NA,[5877454],603,0.003875,[5877454],603,0.003875,NA,NA,NA,26832.09,0.004227,0,26832.09,26832.09,0
8,[],NA,NA,[5560756],117,0.000752,[5560756],117,0.000752,NA,NA,NA,24865.44,0.003917,0,24865.44,24865.44,0
9,[],NA,NA,[89343],79,0.000508,[89343],79,0.000508,NA,NA,NA,24149.61,0.003804,0,24149.61,24149.61,0


In [58]:
rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)


rules_labeled = pd.merge(rules_exploded,unique_items, how='inner', left_on = 'antecedent&consequent', right_on = 'product_id').sort_values("rules_index").reset_index(drop=True)


rules_labeled.to_excel("rules_labeled.xlsx",index=False)

#rules_labeled = rules_labeled.drop('rules_index', axis=1)
rules_labeled


rules_labeled_new = rules_labeled[["rules_index","antecedent&consequent","sup_ant&cons","category_id","brand","price"]]

HTML(rules_labeled_new.to_html(index=False))


rules_index,antecedent&consequent,sup_ant&cons,category_id,brand,price
0,5560754,365,1487580006300255120,strong,194.44
1,5809910,7458,1602943681873052386,grattol,5.24
2,5751422,3457,1487580005268456287,uno,10.95
3,5751383,2912,1487580005092295511,uno,10.32
4,5850281,209,1487580006300255120,marathon,137.78
5,5849033,2753,1487580005092295511,uno,10.32
6,5792800,2683,1487580005268456287,nan,10.32
7,5877454,603,1487580006300255120,jessnail,44.29
8,5560756,117,1487580006300255120,strong,207.94
9,89343,79,2193074740619379535,nan,299.81


#### 3.2 Profit based Association Rules (Limitations on transaction length as at least 2)

In [66]:
import modified_fp_growth_algorithm.modified_fp_growth_latest as mod_fp_growth

#For better trackingI take price instead of profit as profit, because tracking ius easier and at moment I anyway just multiply by 0.1 for profit
# We see how "super efficient it is, only 6.8 seconds for 13k rules inclduing writing to Excel"

data = data[data["product_id"].apply(len)<20]

rules = mod_fp_growth.fpgrowthFromDataFrame(\
    data,
    minSupRatio=0.001,
    maxSupRatio=1,
    minTransactionLength=2,
    maxTransactionLength=10,
    minConf=0,
    item_col=1,
    profit_col=3,
    max_profit = max_profit,
    profit_sensitivity = lambda x : 1 * x
    )
    
rules.to_excel("fp_groth_out.xlsx",index=False) 
rules

,antecedent,sup_antecedent,percSupAntecedent,consequent,sup_consequent,percSupConsequent,antecedent&consequent,sup_ant&cons,sup_perc_ant&cons,confidence,lift,improvement,profit_associated,perc_of_total_profit,profit_associated_prev,net_change,profit_last_item,loss_by_change
0,[5751422],2860,0.019934,[5751383],2441,0.017014,"[5751383, 5751422]",837,0.005834,0.292657,17.201082,0.275643,17793.712298,0.003597,31344.830000,-13551.117702,8620.417644,-22171.535346
1,[5809910],6244,0.043521,[5809912],2596,0.018094,"[5809910, 5809912]",1180,0.008225,0.188981,10.444281,0.170887,12310.167198,0.002489,32651.160000,-20340.992802,6139.704545,-26480.697348
2,[5751422],2860,0.019934,[5849033],2288,0.015947,"[5849033, 5751422]",529,0.003687,0.184965,11.598391,0.169018,11234.717035,0.002271,31344.830000,-20110.112965,5437.019458,-25547.132423
3,[5809910],6244,0.043521,[5809911],1412,0.009842,"[5809911, 5809910]",756,0.005269,0.121076,12.302357,0.111235,7904.139745,0.001598,32651.160000,-24747.020255,3950.860283,-28697.880538
4,[4185],422,0.002941,[3762],282,0.001966,"[3762, 4185]",178,0.001241,0.421801,214.596467,0.419835,6873.959977,0.001390,8167.550000,-1293.590023,3428.879645,-4722.469668
5,[5528035],1596,0.011124,[5528034],750,0.005228,"[5528034, 5528035]",340,0.002370,0.213033,40.751997,0.207805,6463.594661,0.001307,15127.440000,-8663.845339,3240.957067,-11904.802406
6,"[5809910, 5809912]",1180,0.008225,[5809911],1412,0.009842,"[5809911, 5809910, 5809912]",381,0.002656,0.322881,32.807444,0.313040,5965.831635,0.001206,12310.167198,-6344.335562,1991.108159,-8335.443721
7,[5809910],6244,0.043521,[5816170],1398,0.009744,"[5816170, 5809910]",569,0.003966,0.091127,9.352039,0.081383,5955.622664,0.001204,32651.160000,-26695.537336,2980.204657,-29675.741992
8,[5823970],465,0.003241,[5820270],413,0.002879,"[5820270, 5823970]",107,0.000746,0.230108,79.936458,0.227229,5734.939689,0.001159,12493.430000,-6758.490311,2860.107409,-9618.597720
9,[5809912],2596,0.018094,[5809911],1412,0.009842,"[5809911, 5809912]",546,0.003806,0.210324,21.370633,0.200482,5694.313231,0.001151,13507.350000,-7813.036769,2853.399093,-10666.435863


In [68]:
# A minSupRatio of 0.007 means, in case of profitability, that it represents 0.007 of total revenues rspectively profit. We made the profit fo each product equal.
# Let's oberserve what kind of items we have found.


rules_exploded = rules.explode("antecedent&consequent")[["antecedent&consequent","sup_ant&cons","profit_associated","perc_of_total_profit"]]
rules_exploded.insert(0,"rules_index",rules_exploded.index)

rules_labeled =pd.merge(rules_exploded,unique_items, how="inner", left_on="antecedent&consequent",right_on="product_id").sort_values("rules_index").reset_index(drop=True)

rules_labeled.to_excel("rules_labeled.xlsx",index=False)

rules_labeled["profit_associated"] = rules_labeled["profit_associated"].astype(int)

rules_labeled_new = rules_labeled[["rules_index","antecedent&consequent","sup_ant&cons","category_id","brand","price"]]

HTML(rules_labeled_new.to_html(index=False))

rules_index,antecedent&consequent,sup_ant&cons,category_id,brand,price
0,5751383,837,1487580005092295511,uno,10.32
0,5751422,837,1487580005268456287,uno,10.95
1,5809912,1180,1602943681873052386,grattol,5.24
1,5809910,1180,1602943681873052386,grattol,5.24
2,5849033,529,1487580005092295511,uno,10.32
2,5751422,529,1487580005268456287,uno,10.95
3,5809911,756,1602943681873052386,grattol,5.24
3,5809910,756,1602943681873052386,grattol,5.24
4,3762,178,1487580005411062629,cnd,19.37
4,4185,178,1487580005411062629,cnd,19.37
